# Init data


Lancer la commande `docker cp allData.zip jupyter:/home/jovyan/` en premier lieu.

In [2]:
# Commandes à lancer dans le notebook (WSL & Mac) 
!ls
!unzip -o allData.zip 
!ls

allData.zip  sample.csv  work
Archive:  allData.zip
   creating: allData/
  inflating: allData/160109-histoire.txt  
  inflating: allData/160109-intégration.txt  
  inflating: allData/160109-livre.txt  
  inflating: allData/160109-pollution.txt  
  inflating: allData/160109-RoyaumeUni.txt  
  inflating: allData/160109-société.txt  
  inflating: allData/160109-territoires.txt  
  inflating: allData/160524-lobbying.txt  
  inflating: allData/160524-penurie.txt  
  inflating: allData/160524-recherche.txt  
  inflating: allData/160527-agriculture.txt  
  inflating: allData/160527-fonctionnaires.txt  
  inflating: allData/160527-nucleaire.txt  
  inflating: allData/160527-superieur.txt  
  inflating: allData/160602-biodiversite.txt  
  inflating: allData/160602-bloctel.txt  
  inflating: allData/160602-initiatives.txt  
  inflating: allData/160602-innovation.txt  
  inflating: allData/160602-Macron.txt  
  inflating: allData/160602-meteo.txt  
  inflating: allData/160602-superieur.txt  


In [3]:
minio_ip_address = "minio"

In [4]:
from minio import Minio
client_minio = Minio(
    f"{minio_ip_address}:9000",
    access_key="root",
    secret_key="password",
    secure=False
)

# Création du bucket tp6
if client_minio.bucket_exists("tp7") == False:
    client_minio.make_bucket("tp7")



Remise à zéro : destruction des fichiers allData de Minio (si présents)

In [5]:
# Clear directory
directory = "allData"
from minio.deleteobjects import DeleteObject
def deleteFolder1(bucketname, folderName):
    # Delete using "remove_objects"
    objects_to_delete = client_minio.list_objects(bucketname, prefix=folderName, recursive=True)
    objects_to_delete = [DeleteObject(x.object_name) for x in objects_to_delete]
    print(objects_to_delete)
    for del_err in client_minio.remove_objects(bucketname, objects_to_delete):
        print("Deletion Error: {}".format(del_err))
deleteFolder1("tp7", "allData")


[]


Copie des fichiers allData dans [Minio](http://127.0.0.1:19001/tp7) (10 par seconde)

In [6]:
import time, os
for f in os.listdir(directory):
    client_minio.fput_object("tp7", f"{directory}/{f}", f"{directory}/{f}")
    time.sleep(0.1)